In [1]:
# Imports and file paths
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Paths to uploaded files (already present in the Colab environment)
NETM_PATH = '/content/drive/MyDrive/DMV PROJECT/Datasets/Global Migration Flows + Economic Indicators/API_SM.POP.NETM_DS2_en_csv_v2_242491.csv'
GDP_PATH  = '/content/drive/MyDrive/DMV PROJECT/Datasets/Global Migration Flows + Economic Indicators/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_254306/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_254306.csv'

In [3]:
# Helper to read World Bank-style CSVs (skip 4 header rows)
def read_worldbank_csv(path):
    df = pd.read_csv(path, skiprows=4)
    return df

df_netm = read_worldbank_csv(NETM_PATH)
df_gdp  = read_worldbank_csv(GDP_PATH)

print('NetM shape:', df_netm.shape)
print('GDP shape :', df_gdp.shape)

# Inspect columns (first 12)
print('\nNetM columns sample:', df_netm.columns[:12].tolist())


NetM shape: (266, 70)
GDP shape : (266, 70)

NetM columns sample: ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967']


In [4]:
# Identify ID columns and year columns
id_cols = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']
year_cols_netm = [c for c in df_netm.columns if c not in id_cols]
year_cols_gdp  = [c for c in df_gdp.columns  if c not in id_cols]

# Melt to long
netm_long = df_netm.melt(id_vars=['Country Name','Country Code'],
                         value_vars=year_cols_netm, var_name='Year', value_name='NetMigration')

gdp_long = df_gdp.melt(id_vars=['Country Name','Country Code'],
                       value_vars=year_cols_gdp, var_name='Year', value_name='GDP_USD')

# Convert Year to numeric
netm_long['Year'] = pd.to_numeric(netm_long['Year'], errors='coerce')
gdp_long['Year']  = pd.to_numeric(gdp_long['Year'], errors='coerce')

# Convert values to numeric
netm_long['NetMigration'] = pd.to_numeric(netm_long['NetMigration'], errors='coerce')
gdp_long['GDP_USD']       = pd.to_numeric(gdp_long['GDP_USD'], errors='coerce')

print('netm_long:', netm_long.shape)
print('gdp_long :', gdp_long.shape)

netm_long.head()


netm_long: (17556, 4)
gdp_long : (17556, 4)


,Country Name,Country Code,Year,NetMigration
0,Aruba,ABW,1960.0,-788.0
1,Africa Eastern and Southern,AFE,1960.0,-102704.0
2,Afghanistan,AFG,1960.0,2606.0
3,Africa Western and Central,AFW,1960.0,-759.0
4,Angola,AGO,1960.0,-40935.0


In [5]:
# Remove aggregates and non-country entries.
# This list includes common World Bank aggregates. You can expand it if needed.
aggregates = [
    "World", "High income", "Low income", "Upper middle income", "Lower middle income",
    "OECD members", "Euro area", "European Union", "Arab World",
    "East Asia & Pacific", "Sub-Saharan Africa", "Not classified",
    "Latin America & Caribbean", "Middle East & North Africa",
    "South Asia", "North America", "Fragile and conflict affected situations",
    "IDA total", "IDA & IBRD total"
]

def remove_aggregates(df):
    return df[~df['Country Name'].isin(aggregates)].copy()

netm_clean = remove_aggregates(netm_long)
gdp_clean  = remove_aggregates(gdp_long)

print('After removing aggregates:')
print(' - netm_clean:', netm_clean.shape)
print(' - gdp_clean :', gdp_clean.shape)


After removing aggregates:
 - netm_clean: (16368, 4)
 - gdp_clean : (16368, 4)


In [6]:
# --- Merge and clean, then save to Drive (Colab) ---
import os

# (1) Merge on Country Code + Year
merged = pd.merge(netm_clean, gdp_clean, on=['Country Code','Year'], how='outer', suffixes=('_netm','_gdp'))

# (2) Keep a single Country Name (prefer netm)
merged['Country Name'] = merged['Country Name_netm'].fillna(merged['Country Name_gdp'])

# (3) Drop intermediate name columns
merged = merged.drop(columns=['Country Name_netm','Country Name_gdp'], errors='ignore')

# Report counts BEFORE cleaning
print('Before cleaning:')
print(' - merged shape:', merged.shape)
print(' - rows with Year missing:', merged['Year'].isna().sum())
print(' - rows where both NetMigration and GDP_USD are NA:', ((merged['NetMigration'].isna()) & (merged['GDP_USD'].isna())).sum())

# (4) Remove rows where Year is missing (these are header/artifact rows)
rows_with_year = merged.shape[0]
merged = merged.dropna(subset=['Year']).copy()
rows_after_year_drop = merged.shape[0]
removed_year_rows = rows_with_year - rows_after_year_drop
print(f'Removed {removed_year_rows} rows with missing Year.')

# OPTIONAL: If you also want to remove rows that have neither NetMigration nor GDP (purely empty),
# uncomment the next lines. This is stricter and will remove rows with no useful numeric data.
# rows_before_optional = merged.shape[0]
# merged = merged[~(merged['NetMigration'].isna() & merged['GDP_USD'].isna())].copy()
# rows_after_optional = merged.shape[0]
# removed_optional = rows_before_optional - rows_after_optional
# print(f'Additionally removed {removed_optional} rows with both NetMigration and GDP_USD missing.')

# (5) Final sanity prints
merged['Year'] = pd.to_numeric(merged['Year'], errors='coerce')  # ensure numeric year
print('After cleaning:')
print(' - merged shape:', merged.shape)
print(' - rows with Year missing (should be 0):', merged['Year'].isna().sum())

# (6) Save to Google Drive (ensure Drive mounted and folder exists)
drive_folder = '/content/drive/MyDrive/DMV PROJECT'
os.makedirs(drive_folder, exist_ok=True)
out_path = os.path.join(drive_folder, 'Final_merged_migration_gdp_clean.csv')
merged.to_csv(out_path, index=False)
print('Saved cleaned merged file to:', out_path)


Before cleaning:
 - merged shape: (16368, 5)
 - rows with Year missing: 248
 - rows where both NetMigration and GDP_USD are NA: 248
Removed 248 rows with missing Year.
After cleaning:
 - merged shape: (16120, 5)
 - rows with Year missing (should be 0): 0
Saved cleaned merged file to: /content/drive/MyDrive/DMV PROJECT/Final_merged_migration_gdp_clean.csv


In [7]:
merged.describe()

,Year,NetMigration,GDP_USD
count,16120.000000,1.612000e+04,1.344400e+04
mean,1992.000000,-5.474311e+04,6.361461e+11
std,18.762245,4.690620e+05,3.279391e+12
min,1960.000000,-1.009204e+07,2.585956e+06
25%,1976.000000,-2.322300e+04,1.986460e+09
50%,1992.000000,-1.065500e+03,1.297779e+10
75%,2008.000000,5.318000e+03,1.205482e+11
max,2024.000000,4.643836e+06,6.304064e+13


In [8]:
# Mising values
merged.isnull().sum()

,0
Country Code,0
Year,0
NetMigration,0
GDP_USD,2676
Country Name,0


In [9]:
missing_counts = (
    merged.groupby('Country Name')['GDP_USD']
      .apply(lambda s: s.isna().sum())
      .sort_values(ascending=False)
)

print("Top 20 countries with missing GDP data:")
print(missing_counts.head(20))

Top 20 countries with missing GDP data:
Country Name
British Virgin Islands       65
Gibraltar                    65
Korea, Dem. People's Rep.    65
St. Martin (French part)     61
South Sudan                  57
Sint Maarten (Dutch part)    49
Kosovo                       48
Cayman Islands               47
Eritrea                      45
Northern Mariana Islands     44
American Samoa               44
Virgin Islands (U.S.)        44
Guam                         44
Curacao                      41
Turks and Caicos Islands     41
Afghanistan                  41
Channel Islands              40
San Marino                   39
Montenegro                   37
Yemen, Rep.                  36
Name: GDP_USD, dtype: int64


In [10]:
# 2. Identify countries to drop (>= 46 missing values)
countries_to_drop = missing_counts[missing_counts >= 46].index

# 3. Filter the dataframe
merged_filtered = merged[~merged['Country Name'].isin(countries_to_drop)].copy()

# Summary of what happened
print(f"\nDropped {len(countries_to_drop)} countries: {list(countries_to_drop)}")
print(f"Original Row Count: {len(merged)}")
print(f"New Row Count: {len(merged_filtered)}")
print(merged_filtered.isnull().sum())


Dropped 8 countries: ['British Virgin Islands', 'Gibraltar', "Korea, Dem. People's Rep.", 'St. Martin (French part)', 'South Sudan', 'Sint Maarten (Dutch part)', 'Kosovo', 'Cayman Islands']
Original Row Count: 16120
New Row Count: 15600
Country Code       0
Year               0
NetMigration       0
GDP_USD         2219
Country Name       0
dtype: int64


In [11]:
# Ensure Year is numeric
merged_filtered['Year'] = pd.to_numeric(merged_filtered['Year'], errors='coerce')

# Count missing GDP per year
missing_by_year = (
    merged_filtered.groupby('Year')['GDP_USD']
      .apply(lambda s: s.isna().sum())
      .sort_values(ascending=False)
)

# Print top 20 years
print("Top 20 years with most missing GDP values:\n")
print(missing_by_year.head(40))

Top 20 years with most missing GDP values:

Year
1960.0    104
1961.0    102
1962.0    100
1963.0    100
1964.0    100
1965.0     94
1966.0     93
1967.0     90
1968.0     89
1969.0     89
1970.0     67
1971.0     66
1972.0     66
1973.0     66
1974.0     66
1975.0     62
1976.0     62
1978.0     60
1977.0     59
1979.0     59
1980.0     49
1981.0     49
1982.0     48
1983.0     48
1984.0     46
1985.0     44
1986.0     44
1987.0     37
1988.0     34
1989.0     34
2024.0     28
1990.0     17
1991.0     16
1992.0     15
1993.0     14
1994.0     13
2023.0     12
1995.0     10
1996.0     10
1997.0      8
Name: GDP_USD, dtype: int64


In [12]:
# Ensure Year is numeric
merged_filtered['Year'] = pd.to_numeric(merged_filtered['Year'], errors='coerce')

# Count missing GDP per year
missing_by_year = (
    merged_filtered.groupby('Year')['GDP_USD']
        .apply(lambda s: s.isna().sum())
        .sort_values(ascending=False)
)

# Identify years to remove
years_to_remove = missing_by_year[missing_by_year >= 52].index.tolist()

print("Years with >= 52 missing GDP values (will be removed):")
print(years_to_remove)

# Remove those years
merged_filtered_years = merged_filtered[~merged_filtered['Year'].isin(years_to_remove)].copy()

print("\nRows before removing years:", len(merged_filtered))
print("Rows after removing years:", len(merged_filtered_years))
print("Years removed:", years_to_remove)
print("Remaining years:", merged_filtered_years['Year'].nunique())
# Save to CSV locally
merged_filtered_years.to_csv("output.csv", index=False)
files.download('output.csv')

Years with >= 52 missing GDP values (will be removed):
[1960.0, 1961.0, 1962.0, 1963.0, 1964.0, 1965.0, 1966.0, 1967.0, 1968.0, 1969.0, 1970.0, 1971.0, 1972.0, 1973.0, 1974.0, 1975.0, 1976.0, 1978.0, 1977.0, 1979.0]

Rows before removing years: 15600
Rows after removing years: 10800
Years removed: [1960.0, 1961.0, 1962.0, 1963.0, 1964.0, 1965.0, 1966.0, 1967.0, 1968.0, 1969.0, 1970.0, 1971.0, 1972.0, 1973.0, 1974.0, 1975.0, 1976.0, 1978.0, 1977.0, 1979.0]
Remaining years: 45


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
# assume merged is your cleaned dataset
mydf = merged_filtered_years.copy()

# ensure numeric
mydf['Year'] = pd.to_numeric(mydf['Year'], errors='coerce')

TARGET_YEAR = 2024

# list of countries you want to impute for
countries_to_impute = [
"Aruba","Afghanistan","American Samoa","Bhutan","Channel Islands","Cuba","Curacao",
"Cayman Islands","Eritrea","Ethiopia","Faroe Islands","Gibraltar","Greenland","Guam",
"Isle of Man","Korea, Rep.","Lebanon","Liechtenstein","St. Martin (French part)",
"Monaco","Northern Mariana Islands","New Caledonia","Palau","Korea, Dem. People's Rep.",
"French Polynesia","San Marino","South Sudan","Syrian Arab Republic","Tonga","Tuvalu",
"Venezuela, RB","British Virgin Islands","Virgin Islands (U.S.)","Yemen, Rep."
]

imputed_rows = []

for cname in countries_to_impute:

    # select country data between 2000–2023
    g = mydf[(mydf['Country Name'] == cname) &
           (mydf['Year'] >= 2000) &
           (mydf['Year'] <= 2023)][['Year','GDP_USD']].dropna()

    # need at least 2 points to compute slope
    if len(g) < 2:
        continue

    # linear regression (simple: slope & intercept)
    x = g['Year'].values
    y = g['GDP_USD'].values

    a, b = np.polyfit(x, y, 1)         # slope a, intercept b
    pred_2024 = a * TARGET_YEAR + b    # predicted GDP for 2024

    # write into df ONLY IF 2024 exists and is NaN
    mask = (mydf['Country Name'] == cname) & (mydf['Year'] == TARGET_YEAR) & (mydf['GDP_USD'].isna())
    mydf.loc[mask, 'GDP_USD'] = pred_2024

    imputed_rows.append([cname, len(g), pred_2024])

# show what was filled
imputed_df = pd.DataFrame(imputed_rows, columns=['Country Name','Points Used','Predicted 2024 GDP'])
print(imputed_df)

# save the updated dataset
mydf.to_csv('slope_dataframe.csv', index=False)
files.download('slope_dataframe.csv')

                Country Name  Points Used  Predicted 2024 GDP
0                      Aruba           24        3.420800e+09
1                Afghanistan           24        2.250473e+10
2             American Samoa           21        7.773013e+08
3                     Bhutan           24        3.229682e+09
4            Channel Islands           23        1.149671e+10
5                       Cuba           21        1.228372e+11
6                    Curacao           24        3.267253e+09
7                    Eritrea           12        3.401172e+09
8                   Ethiopia           23        1.183078e+11
9              Faroe Islands           24        3.861596e+09
10                 Greenland           24        3.566107e+09
11                      Guam           21        7.025169e+09
12               Isle of Man           23        9.300576e+09
13               Korea, Rep.           24        1.930288e+12
14                   Lebanon           24        4.491789e+10
15      

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
mydf.head(), mydf.columns

(   Country Code    Year  NetMigration  GDP_USD Country Name
 20          ABW  1980.0        -756.0      NaN        Aruba
 21          ABW  1981.0        -547.0      NaN        Aruba
 22          ABW  1982.0        -413.0      NaN        Aruba
 23          ABW  1983.0        -405.0      NaN        Aruba
 24          ABW  1984.0        -535.0      NaN        Aruba,
 Index(['Country Code', 'Year', 'NetMigration', 'GDP_USD', 'Country Name'], dtype='object'))

In [15]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# --- ASSUMPTIONS ---
# Data is loaded into 'mydf' with columns: 'Country Name', 'Year', 'GDP_USD', 'NetMigration'

# --- Configuration ---
COUNTRY_COL = 'Country Name'
YEAR_COL = 'Year'
GDP_COL = 'GDP_USD'
NETM_COL = 'NetMigration'
MAX_COUNTRIES = 40 # Limit for dropdown to keep it manageable

# --- Helper for formatting hover text (Simplified) ---
def format_money(x):
    if pd.isna(x) or x <= 0:
        return "NA"
    return f"${x:,.0f}"

def format_int(x):
    if pd.isna(x):
        return "NA"
    return f"{int(x):,}"

# --- Data Preparation ---
data = mydf.copy()

# Ensure Year is integer
data[YEAR_COL] = pd.to_numeric(data[YEAR_COL], errors='coerce').astype('Int64')
data.dropna(subset=[YEAR_COL], inplace=True)
data[YEAR_COL] = data[YEAR_COL].astype(int)

# Use positive GDP for log axis
data['GDP_pos'] = data[GDP_COL].where(data[GDP_COL] > 0)

# Calculate bubble size based on absolute net migration
# Formula: sqrt(|NetMigration| + 1) * scaling_factor
data['bubble_size'] = np.sqrt(data[NETM_COL].abs().fillna(0) + 1) * 4

# Identify years and top countries for the dropdown
years = sorted(data[YEAR_COL].unique())
latest_year = data[YEAR_COL].max()
latest_data = data[data[YEAR_COL] == latest_year].sort_values('GDP_pos', ascending=False)
countries = latest_data[COUNTRY_COL].dropna().unique()[:MAX_COUNTRIES]

# Filter the main data to only include the selected countries
data = data[data[COUNTRY_COL].isin(countries)]

# Create hover text
data['hover_text'] = data.apply(
    lambda row: f"<b>{row[COUNTRY_COL]}</b><br>Year: {row[YEAR_COL]}<br>GDP: {format_money(row['GDP_pos'])}<br>Net Migration: {format_int(row[NETM_COL])}",
    axis=1
)

# --- Compute Global Axis Ranges and SizeRef (for fixed scale) ---

# GDP (log axis) - Calculate fixed range based on all positive GDP values
gdp_pos = data['GDP_pos'].dropna()
log_gmin = np.log10(gdp_pos.min())
log_gmax = np.log10(gdp_pos.max())
xaxis_range_log = [log_gmin - 0.35, log_gmax + 0.35]

# Net Migration (linear) - Calculate fixed range
net_vals = data[NETM_COL].dropna()
nmin, nmax = net_vals.min(), net_vals.max()
y_pad = max(1.0, 0.15 * max(abs(nmin), abs(nmax)))
yaxis_range = [nmin - y_pad, nmax + y_pad]

# Size reference for consistent bubble sizing
max_bubble = data['bubble_size'].max()
desired_max_px = 60
sizeref = 2.0 * max_bubble / (desired_max_px ** 2)


# --- Build Figure (using go for custom animation) ---

# Generate a color map for countries
palette = px.colors.qualitative.Set2
color_map = {c: palette[i % len(palette)] for i, c in enumerate(countries)}

# Pre-group data by year for efficient frame generation
data_by_year = {y: data[data[YEAR_COL] == y] for y in years}

# 1. Base Traces (Markers + Lines)
base_traces = []
initial_year = years[0]
for c in countries:
    # Find initial point (may be NaN)
    initial_point = data[(data[COUNTRY_COL] == c) & (data[YEAR_COL] == initial_year)]

    # Use first available data if initial year is missing
    if initial_point.empty:
        initial_point = data[data[COUNTRY_COL] == c].sort_values(YEAR_COL).head(1)

    # Extract initial point data
    x0 = initial_point['GDP_pos'].tolist()
    y0 = initial_point[NETM_COL].tolist()
    size0 = initial_point['bubble_size'].tolist()
    hover0 = initial_point['hover_text'].iloc[0] if not initial_point.empty else f"<b>{c}</b><br>Year: {initial_year} (no data)"

    country_color = color_map[c]

    # Marker Trace (The bubble for the current year)
    t_marker = go.Scatter(
        x=x0, y=y0, mode='markers', name=c,
        marker=dict(size=size0, sizemode='area', sizeref=sizeref, sizemin=4,
                    line=dict(width=0.5, color='rgba(0,0,0,0.3)'), opacity=0.9, color=country_color),
        hoverinfo='text', hovertext=hover0, visible=True
    )

    # Line Trace (The history path)
    t_line = go.Scatter(
        x=x0, y=y0, mode='lines', name=f"{c} path",
        line=dict(width=2, color=country_color),
        hoverinfo='skip',
        visible=False,   # Hide lines by default (Only show when one country is selected)
        showlegend=False
    )

    base_traces.extend([t_marker, t_line])

# 2. Frames (The year-by-year updates for animation)
frames = []
for y in years:
    frame_data = []
    # Collect all traces for this year
    for c_idx, c in enumerate(countries):
        country_data = data_by_year[y][data_by_year[y][COUNTRY_COL] == c]

        # Current bubble point
        x_curr = country_data['GDP_pos'].tolist()
        y_curr = country_data[NETM_COL].tolist()
        size_curr = country_data['bubble_size'].tolist()
        hover_curr = country_data['hover_text'].tolist()
        if not x_curr: # Handle missing data for the year
            x_curr = [None]
            y_curr = [None]
            size_curr = [0]
            hover_curr = [f"<b>{c}</b><br>Year: {y} (no data)"]

        marker_dict = dict(size=size_curr, sizemode='area', sizeref=sizeref, sizemin=4,
                           color=color_map[c], line=dict(width=0.5, color='rgba(0,0,0,0.3)'), opacity=0.9)
        trace_marker = dict(type='scatter', x=x_curr, y=y_curr, mode='markers',
                            marker=marker_dict, hovertext=hover_curr, hoverinfo='text')

        # History path for this year (cumulative up to y)
        hist_data = data[(data[COUNTRY_COL] == c) & (data[YEAR_COL] <= y)].sort_values(YEAR_COL)
        x_hist = hist_data['GDP_pos'].tolist()
        y_hist = hist_data[NETM_COL].tolist()

        trace_line = dict(type='scatter', x=x_hist, y=y_hist, mode='lines',
                          line=dict(width=2, color=color_map[c]), hoverinfo='skip', showlegend=False)

        frame_data.extend([trace_marker, trace_line])

    frames.append(go.Frame(data=frame_data, name=str(y), layout=go.Layout(title_text=f'Year: {y}')))


# 3. Dropdown Menu Setup (to show all or one country's line trace)
total_traces = len(base_traces)
dropdown_buttons = []

# "All" button: Markers visible, Lines invisible
visible_all = [True if i % 2 == 0 else False for i in range(total_traces)]
dropdown_buttons.append(dict(
    label='All Countries',
    method='update',
    args=[{'visible': visible_all}, {'title': f'Global Economic–Migration Bubble Map — Year: {initial_year}'}]
))

# Per-country buttons: Only that country's marker AND line visible
for i, c in enumerate(countries):
    vis = [False] * total_traces
    vis[i * 2] = True       # Marker index (even)
    vis[i * 2 + 1] = True   # Line index (odd)
    dropdown_buttons.append(dict(
        label=c,
        method='update',
        args=[{'visible': vis}, {'title': f'{c} — GDP vs Net Migration (Year: {initial_year})'}]
    ))

updatemenus = [
    dict(buttons=dropdown_buttons, direction='down', showactive=True, x=0.01, xanchor='left', y=1.12, yanchor='top',
         bgcolor='rgba(255,255,255,0.95)', font=dict(size=12))
]


# 4. Layout and Controls
layout = go.Layout(
    template='plotly_white',
    title=f'Global Economic–Migration Bubble Map — Year: {initial_year}',
    xaxis=dict(
        title='GDP (USD, log scale)', type='log', range=xaxis_range_log, tickformat='.2s'
    ),
    yaxis=dict(
        title='Net Migration', range=yaxis_range
    ),
    height=720,
    hovermode='closest',
    updatemenus=updatemenus + [
        dict(type='buttons', showactive=False, y=0.08, x=0.01, xanchor='left',
             buttons=[
                 dict(label='Play', method='animate', args=[None, {"frame": {"duration": 450, "redraw": True}, "transition": {"duration": 250}}]),
                 dict(label='Pause', method='animate', args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}}])
             ])
    ],
    legend=dict(title='Region / Country', x=1.02, y=1),
    margin=dict(l=90, r=260, t=140, b=100),
    plot_bgcolor='rgba(255,255,255,0.98)'
)

# Slider linked to frames
sliders = [dict(
    steps=[dict(method='animate', args=[[str(y)], {"frame": {"duration": 450, "redraw": True}, "transition": {"duration": 250}}], label=str(y)) for y in years],
    active=0,
    currentvalue=dict(prefix="Year: ", font=dict(size=16)),
    pad=dict(t=50)
)]

# 5. Create and Display Figure
fig = go.Figure(data=base_traces, layout=layout, frames=frames)
fig.update_layout(sliders=sliders)
fig.update_traces(hoverlabel=dict(bgcolor='white', font_size=12))
fig.show()

In [16]:
from google.colab import output
output.enable_custom_widget_manager()

!git config --global user.email "gnandeeoboppudi@gmail.com"
!git config --global user.name "gnandeep-2002"

In [18]:
!git clone https://github.com/gnandeep-2002/Global-Migration-and-Economic-Indicator-Visualization-Using-Bubble-Chart

Cloning into 'Global-Migration-and-Economic-Indicator-Visualization-Using-Bubble-Chart'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), done.


In [21]:
!cp /content/DMV_PROJECT.ipynb "/content/Global-Migration-and-Economic-Indicator-Visualization-Using-Bubble-Chart/"

cp: cannot stat '/content/DMV_PROJECT.ipynb': No such file or directory
